<a href="https://colab.research.google.com/github/vorad1/CS703_dvor451/blob/working/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from huggingface_hub import login
import os

# Get the secret
token = HF_TOKEN
login(token=token)

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

# Loading FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up inference pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Load CommonsenseQA dataset
dataset = load_dataset("commonsense_qa")


Device set to use cpu


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.